## MNIST

In [0]:
import warnings
warnings.filterwarnings(action = 'ignore')
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [0]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

W0825 04:10:55.362333 139777173350272 deprecation.py:323] From <ipython-input-2-71e12f4bac70>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0825 04:10:55.364948 139777173350272 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0825 04:10:55.369598 139777173350272 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz


W0825 04:10:55.774033 139777173350272 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0825 04:10:55.776736 139777173350272 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0825 04:10:55.870932 139777173350272 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be remove

Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
# hypter parameter setting
learning_rate = 1e-4
training_epochs = 10
display_step = 1
batch_size = 32
activation = tf.nn.relu

In [0]:
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def bias_variable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

tf_X = tf.placeholder("float", shape=[None, 784])
tf_X_reshape = tf.reshape(tf_X, [-1,28,28,1])
tf_Y = tf.placeholder("float", shape=[None, 10])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(tf_X_reshape, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

pred=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

loss = -tf.reduce_sum(tf_Y*tf.log(pred))
optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)
correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(tf_Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

W0825 05:11:51.177445 139777173350272 deprecation.py:506] From <ipython-input-4-ac002f1e36ef>:36: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(1000):
    batch = mnist.train.next_batch(50)
    sess.run(optimizer,feed_dict={tf_X: batch[0], tf_Y: batch[1], keep_prob: 0.5})

print("test accuracy %g"% sess.run(accuracy, feed_dict={
       tf_X: mnist.test.images, tf_Y: mnist.test.labels, keep_prob: 1.0}))

test accuracy 0.9622


## CIFAR10

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets.cifar10 import load_data

In [0]:
(x_train, y_train), (x_test, y_test) = load_data()

# one-hot encoding
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10),axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10),axis=1)

In [0]:
# hypter parameter setting
learning_rate = 1e-3
training_epochs = 100
display_step = 10
batch_size = 128
activation = tf.nn.tanh

In [0]:
def CNN_model(x):
    x_image = x
    
    #[window size(,),channel, # of features]
    W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 3, 64], stddev=1e-4)) 
    b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1

    h_pool1 = tf.nn.relu(tf.nn.max_pool(h_conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME'))
    h_norm1 = tf.contrib.layers.batch_norm(h_pool1)

    W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 64, 64], stddev=1e-2))
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_norm1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)

    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    h_norm2 = tf.contrib.layers.batch_norm(h_pool2)

    
    W_conv3 = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], stddev=1e-2))
    b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
    h_conv3 = tf.nn.relu(tf.nn.conv2d(h_norm2, W_conv3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3)
    h_pool3 = tf.nn.max_pool(h_conv3, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    h_norm3 = tf.contrib.layers.batch_norm(h_pool3)
    
#     W_conv4 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=1e-2))
#     b_conv4 = tf.Variable(tf.constant(0.1, shape=[128])) 
#     h_conv4 = tf.nn.relu(tf.nn.conv2d(h_pool3, W_conv4, strides=[1, 1, 1, 1], padding='SAME') + b_conv4)

#     W_conv5 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=1e-2))
#     b_conv5 = tf.Variable(tf.constant(0.1, shape=[128]))
#     h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4, W_conv5, strides=[1, 1, 1, 1], padding='SAME') + b_conv5)


    W_fc1 = tf.Variable(tf.truncated_normal(shape=[4 * 4 * 128, 384], stddev=5e-2))
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[384]))

    h_conv5_flat = tf.reshape(h_norm3, [-1, 4*4*128])
    h_fc1 = tf.nn.tanh(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)
    h_fc1_norm = tf.contrib.layers.batch_norm(h_fc1)
    

    h_fc1_drop = tf.nn.dropout(h_fc1_norm, keep_prob)
    
    W_fc2 = tf.Variable(tf.truncated_normal(shape=[384, 1024], stddev=5e-2))
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[1024]))
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
    
    
    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

    W_fc3 = tf.Variable(tf.truncated_normal(shape=[1024, 10], stddev=5e-2))
    b_fc3 = tf.Variable(tf.constant(0.1, shape=[10]))
    logits = tf.matmul(h_fc2_drop,W_fc3) + b_fc3
    y_pred = tf.nn.softmax(logits)

    return y_pred, logits

tf_X = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
tf_Y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

pred, logit = CNN_model(tf_X)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_Y, logits=logit))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(tf_Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [54]:
def next_batch(num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
  
    for i in range(10000):
        batch = next_batch(516, x_train, y_train_one_hot.eval())

        if i % 1000 == 0:
            train_accuracy = accuracy.eval(feed_dict={tf_X: batch[0], tf_Y: batch[1], keep_prob: 0.6})
            loss_print = loss.eval(feed_dict={tf_X: batch[0], tf_Y: batch[1], keep_prob: 0.6})

            print("iteration: %d, accuracy: %f, loss: %f" % (i, train_accuracy, loss_print))
        sess.run(optimizer, feed_dict={tf_X: batch[0], tf_Y: batch[1], keep_prob: 0.6})

    test_accuracy = 0.0  
    for i in range(10):
        test_batch = next_batch(1000, x_test, y_test_one_hot.eval())
        test_accuracy = test_accuracy + accuracy.eval(feed_dict={tf_X: test_batch[0], tf_Y: test_batch[1], keep_prob: 1.0})
    test_accuracy = test_accuracy / 10;
    print("Test accuracy: %f" % test_accuracy)

iteration: 0, accuracy: 0.081395, loss: 3.131934
iteration: 1000, accuracy: 0.825581, loss: 0.471749
iteration: 2000, accuracy: 0.914729, loss: 0.232910
iteration: 3000, accuracy: 0.945736, loss: 0.159654
iteration: 4000, accuracy: 0.967054, loss: 0.099239
iteration: 5000, accuracy: 0.961240, loss: 0.108511
iteration: 6000, accuracy: 0.982558, loss: 0.074417
iteration: 7000, accuracy: 0.961240, loss: 0.115581
iteration: 8000, accuracy: 0.986434, loss: 0.037019
iteration: 9000, accuracy: 0.978682, loss: 0.050396
Test accuracy: 0.800200
